In [8]:
import glob
import os
os.environ['allow_dangerous_deserialization'] = 'true'

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationChain

In [9]:
data_path = "../data/*.txt"
faiss_database = "../data/faiss_index"
file_list = glob.glob(data_path)

In [10]:
def load_data(file):
  # 1️⃣ Carregar documentos (pode substituir pelo seu próprio dataset)
  loader = TextLoader(file)  # Carregue um arquivo local
  documents = loader.load()
  
  return documents

In [ ]:
# 3️⃣ Criar embeddings com Ollama
embeddings = OllamaEmbeddings(model="mistral")  # Usa o modelo "mistral" para embeddings
vectorstore = None
new_vecstore = None
if os.path.exists(faiss_database):
  vectorstore = FAISS.load_local(faiss_database, embeddings, allow_dangerous_deserialization=True)
  print(f'Vecstore loaded from {faiss_database}')
  
lines = []
for file in file_list:
  with open(file, 'r') as f:
    lines.extend(f.readlines())
  os.rename(file, file.replace('data', 'data/indexed_files'))

if len(lines) > 0:    
  with open('.temp', 'w') as f_out:
    f_out.writelines(lines)  
  # 1️⃣ Carregar documentos (pode substituir pelo seu próprio dataset)
  loader = TextLoader(".temp")  # Carregue um arquivo local
  documents = loader.load()

  # 2️⃣ Dividir o texto em pedaços menores para indexação eficiente
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  texts = text_splitter.split_documents(documents)

  # 3️⃣ Criar embeddings com Ollama
  embeddings = OllamaEmbeddings(model="mistral")  # Usa o modelo "mistral" para embeddings

  # 4️⃣ Criar e armazenar a base de dados vetorial com FAISS
  new_vecstore = FAISS.from_documents(texts, embeddings)

if (vectorstore is not None) and (new_vecstore is not None):
  vectorstore.merge_from(new_vecstore)
  vectorstore.save_local(faiss_database)
  print(f'{faiss_database} Updated with new files!!!')
elif new_vecstore is not None and vectorstore is None:
  # Save FAISS index locally
  new_vecstore.save_local(faiss_database)  
  vectorstore = new_vecstore  
  print(f'{faiss_database} Loaded!!!')
elif (vectorstore is None) and (new_vecstore is None):
  raise Exception('No previous vector db found and no new files do vectorize!')

In [12]:
# 5️⃣ Carregar o modelo Llama 3.1 com Ollama para geração de texto
#llm = OllamaLLM(model='llama3.1')  
llm = OllamaLLM(model='llama3.1')

In [13]:
# 6️⃣ Criar o sistema de Perguntas e Respostas (RAG)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    verbose=True
)

In [ ]:
from langchain.prompts import PromptTemplate

# Definir um template de prompt em português
template = """
Responda em português, considere que estamos no país Brasil, a seguinte pergunta com base no contexto fornecido. Sempre informe a fonte da informação:

Pergunta: {question}
Contexto: {context}

Resposta em português:
"""

# Criar o PromptTemplate
QA_PROMPT = PromptTemplate(
    template=template,
    input_variables=["question", "context"]
)

# Configurar o RetrievalQA com o prompt personalizado
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_PROMPT},  # Usar o prompt personalizado    
    verbose=True
)


#query = "Quais são os temas tratados nos documentos?"
#query = 'Quais os direitos relacionados a férias?'
query = 'Qual a pena para cidadãos que cometem assassinato?'

# Fazer uma pergunta
response = qa_chain.invoke(query)
#chain = ConversationChain(llm=llm)
#response = chain.invoke(query)
print(response)